In [38]:
import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import warnings
warnings.filterwarnings("ignore")
import math
import os
from datetime import date, timedelta, datetime
import time
from tqdm import tqdm
import seaborn as sns
from scipy import stats
from matplotlib.ticker import MaxNLocator
from matplotlib.backends.backend_pdf import PdfPages
start_time = time.perf_counter()

In [39]:
price_data = pd.read_csv('stockPriceData-2.csv')
# Convert 'Date' to datetime type
price_data['Date'] = pd.to_datetime(price_data['Date'])

top_500 =  price_data.groupby('Date', group_keys= False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
# Filter price data for dates after '2006-01-01' and drop duplicates
price_data.drop_duplicates(['Date', 'Symbol'], inplace=True)

company_master = pd.read_csv('Companymaster_6_12_2025.csv')

yearly = pd.read_csv('Finance_cons_pl_6_12_2025.csv')
Quaterly = pd.read_csv('Quarterly_cons_17_6_2025.csv')
Finance_bs =  pd.read_csv('Finance_cons_bs_6_12_2025.csv')
cashflow =  pd.read_csv('Finance_cons_cf_6_12_2025.csv')
ratios = pd.read_csv('Finance_cons_fr_6_12_2025.csv')

rebal_dates = pd.read_excel('GrowthDateUpdated.xlsx')

rebal_dates["Date"] = pd.to_datetime(rebal_dates["Date"])
rebal=rebal_dates['Date']

Rebalance_Qtr=list(rebal_dates['Quarter'])
Rebalance_Date=list(rebal_dates['Date'])
Qtr_date_dict=dict(zip(Rebalance_Date,Rebalance_Qtr))
date_Qtr_dict=dict(zip(Rebalance_Qtr,Rebalance_Date))
top_500['Quarter']=top_500['Date'].map(Qtr_date_dict)
index_constituents_data= top_500

X = Quaterly[Quaterly['Result_Type'] == 'Q']
print(X.columns)

Index(['Fincode', 'Result_Type', 'NoOfMonths', 'Date_End', 'NET_SALES',
       'INT_ADV', 'INC_INV', 'INT_BAL', 'INT_OTHERS', 'OTHER_INCOME',
       ...
       'exc_foreign_exch_gain', 'exc_other', 'Curr_tax', 'Def_tax',
       'Fringe_benefits', 'Prior_Period_Tax', 'Mat_Credit', 'Other_Tax',
       'flag', 'notes'],
      dtype='object', length=132)


In [40]:
ratios = ratios[['FINCODE','Year_end','Net_Sales_Growth','Adj_PAT_Growth','FCF_Share']]
ratios = pd.merge(ratios, company_master[['FINCODE', 'SYMBOL']], left_on='FINCODE', right_on='FINCODE')

# Ensure data is sorted correctly
ratios = ratios.sort_values(['SYMBOL', 'Year_end'])

# Custom 1-year FCF growth calculation
def calc_fcf_growth(x):
    prev = x.shift(1)
    growth = (x - prev) / prev.abs()
    return growth

# Apply group-wise custom logic
ratios['FCF_Growth'] = ratios.groupby('SYMBOL')['FCF_Share'].transform(calc_fcf_growth)
ratios =  ratios[['FINCODE','SYMBOL','Year_end','Net_Sales_Growth','Adj_PAT_Growth','FCF_Growth']]

In [41]:

yearly = yearly[['FINCODE','Year_end','Operating_profit','Advertising']]
cashflow = cashflow[['FINCODE','Year_end','Cash_from_Operation']]
data = pd.merge(yearly, cashflow, on=['FINCODE','Year_end'])
data = pd.merge(data, company_master[['FINCODE', 'SYMBOL']], left_on='FINCODE', right_on='FINCODE')

# Ensure data is sorted properly
data = data.sort_values(['SYMBOL', 'Year_end'])

# Function to calculate signed percent change
def signed_pct_change(x):
    prev = x.shift(1)
    diff = x - prev
    return np.sign(diff) * (diff.abs() / prev.abs().replace(0, np.nan))

# Apply to each column
for col in ['Operating_profit', 'Advertising', 'Cash_from_Operation']:
    data[f'{col}_Growth'] = data.groupby('SYMBOL')[col].transform(signed_pct_change)
data

,FINCODE,Year_end,Operating_profit,Advertising,Cash_from_Operation,SYMBOL,Operating_profit_Growth,Advertising_Growth,Cash_from_Operation_Growth
30052,207635,200903,1782.41,NaN,24.75,20MICRONS,NaN,NaN,NaN
30053,207635,201003,2192.98,NaN,213.40,20MICRONS,0.230345,NaN,7.622222
30054,207635,201103,2841.64,NaN,23.88,20MICRONS,0.295789,NaN,-0.888097
30055,207635,201203,3824.34,NaN,1274.12,20MICRONS,0.345821,NaN,52.355109
30056,207635,201303,4236.97,NaN,4457.72,20MICRONS,0.107896,NaN,2.498666
...,...,...,...,...,...,...,...,...,...
23479,132410,202503,630.22,NaN,111.90,NaN,NaN,NaN,NaN
32672,218756,202503,9555.58,NaN,-13626.33,NaN,NaN,NaN,NaN
33847,221423,202503,3181.26,102.69,-2316.25,NaN,NaN,NaN,NaN
41425,315811,202503,1285.01,15.65,-593.37,NaN,NaN,NaN,NaN


In [ ]:
data = data[['FINCODE','Year_end','SYMBOL','Operating_profit_Growth','Advertising_Growth','Cash_from_Operation_Growth']]
# Drop SYMBOL from `data` before merge
data = data.drop(columns='SYMBOL')

# Merge with ratios
growth = pd.merge(data, ratios, on=['FINCODE', 'Year_end'])

# Done
growth

,FINCODE,Year_end,Operating_profit_Growth,Advertising_Growth,Cash_from_Operation_Growth,SYMBOL,Net_Sales_Growth,Adj_PAT_Growth,FCF_Growth
0,207635,200903,NaN,NaN,NaN,20MICRONS,0.000000,0.000000,NaN
1,207635,201003,0.230345,NaN,7.622222,20MICRONS,30.609571,449.403560,-0.534407
2,207635,201103,0.295789,NaN,-0.888097,20MICRONS,41.646693,8.496175,-0.245510
3,207635,201203,0.345821,NaN,52.355109,20MICRONS,18.408545,69.932729,-0.523932
4,207635,201303,0.107896,NaN,2.498666,20MICRONS,3.758729,-56.739741,0.866329
...,...,...,...,...,...,...,...,...,...
41802,132410,202503,NaN,NaN,NaN,NaN,-33.247600,0.000000,NaN
41803,218756,202503,NaN,NaN,NaN,NaN,15.008800,-11.578200,NaN
41804,221423,202503,NaN,NaN,NaN,NaN,15.191400,38.992800,NaN
41805,315811,202503,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN


In [44]:
GrowthDate = pd.read_excel('GrowthDateUpdated.xlsx')[['Date', 'Qtr']]
GrowthDate['Qtr'] = GrowthDate['Qtr'].astype('int')
# GrowthDate Mapping  
final_df =  pd.merge(growth, GrowthDate, left_on='Year_end', right_on='Qtr').drop(columns=['Year_end', 'Qtr'])
final_df = final_df.sort_values(by='Date').reset_index(drop=True)
final_df

,FINCODE,Operating_profit_Growth,Advertising_Growth,Cash_from_Operation_Growth,SYMBOL,Net_Sales_Growth,Adj_PAT_Growth,FCF_Growth,Date
0,101425,0.184819,0.235862,1.163546,BBTC,17.828259,-34.905762,-72.400000,2006-05-31
1,100148,0.223063,-0.486403,-0.337255,UFLEX,18.809516,63.313503,0.446339,2006-05-31
2,109715,0.287318,NaN,1.354516,JAYSREETEA,1.202935,60.876942,1.270973,2006-05-31
3,100125,1.199503,0.289430,0.011765,EIDPARRY,29.314746,10.314882,-1.106609,2006-05-31
4,106943,0.232532,0.333878,-0.011928,JBCHEPHARM,29.771339,20.382793,-1.129714,2006-05-31
...,...,...,...,...,...,...,...,...,...
38564,210551,-1.857322,-0.315789,-0.892932,NITESHEST,-73.142500,16.476600,-0.939075,2025-06-02
38565,228347,0.014507,NaN,0.009151,ATGL,11.735200,-1.961000,-2.679150,2025-06-02
38566,117354,0.156248,0.183973,-0.224078,HAVELLS,17.149300,15.697700,-0.311681,2025-06-02
38567,132505,0.492330,0.516464,0.909205,UCOBANK,14.699900,NaN,-0.076985,2025-06-02


In [45]:
price_data['Date'] = pd.to_datetime(price_data['Date'])
final_df['Date'] = pd.to_datetime(final_df['Date'])
final_df.rename(columns={'SYMBOL' : 'Symbol'},inplace=True)
merged_df = pd.merge(final_df[['Date', 'Symbol', 'Operating_profit_Growth','Advertising_Growth','Cash_from_Operation_Growth','Net_Sales_Growth','Adj_PAT_Growth','FCF_Growth']], price_data, on=['Date', 'Symbol'], how='outer')
merged_df =  merged_df.groupby('Date', group_keys= False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
merged_df

,Date,Symbol,Operating_profit_Growth,Advertising_Growth,Cash_from_Operation_Growth,Net_Sales_Growth,Adj_PAT_Growth,FCF_Growth,Open,High,Low,Close,Volume,Mcap
11,1995-06-16,RELIANCE,NaN,NaN,NaN,NaN,NaN,NaN,13.590561,13.812036,13.590561,13.711366,1272550.0,125404.734065
13,1995-06-16,TATASTEEL,NaN,NaN,NaN,NaN,NaN,NaN,13.474803,13.474803,13.194078,13.317597,580400.0,80205.346611
8,1995-06-16,HINDPETRO,NaN,NaN,NaN,NaN,NaN,NaN,35.061728,35.061728,34.671605,34.765432,500.0,67415.040000
5,1995-06-16,GRASIM,NaN,NaN,NaN,NaN,NaN,NaN,129.684348,129.684348,129.505061,129.505061,250.0,47011.311897
3,1995-06-16,BPCL,NaN,NaN,NaN,NaN,NaN,NaN,12.497917,12.497917,12.497917,12.497917,100.0,44992.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7593674,2025-06-13,NSLNISP,NaN,NaN,NaN,NaN,NaN,NaN,39.150000,39.420000,38.610000,39.050000,3150428.0,114440.158443
7592959,2025-06-13,GENUSPOWER,NaN,NaN,NaN,NaN,NaN,NaN,369.950000,382.800000,368.000000,370.500000,2990813.0,112605.359198
7594499,2025-06-13,WESTLIFE,NaN,NaN,NaN,NaN,NaN,NaN,695.000000,718.950000,691.450000,716.050000,115027.0,111658.090948
7592639,2025-06-13,CCL,NaN,NaN,NaN,NaN,NaN,NaN,800.000000,841.050000,799.950000,833.700000,201301.0,111330.230424


In [46]:
merged_df[['Operating_profit_Growth','Advertising_Growth','Cash_from_Operation_Growth','Net_Sales_Growth','Adj_PAT_Growth','FCF_Growth']] = merged_df.groupby('Symbol', group_keys=False)[['Operating_profit_Growth','Advertising_Growth','Cash_from_Operation_Growth','Net_Sales_Growth','Adj_PAT_Growth','FCF_Growth']].apply(lambda x: x.fillna(method='ffill'))
merged_df

,Date,Symbol,Operating_profit_Growth,Advertising_Growth,Cash_from_Operation_Growth,Net_Sales_Growth,Adj_PAT_Growth,FCF_Growth,Open,High,Low,Close,Volume,Mcap
11,1995-06-16,RELIANCE,NaN,NaN,NaN,NaN,NaN,NaN,13.590561,13.812036,13.590561,13.711366,1272550.0,125404.734065
13,1995-06-16,TATASTEEL,NaN,NaN,NaN,NaN,NaN,NaN,13.474803,13.474803,13.194078,13.317597,580400.0,80205.346611
8,1995-06-16,HINDPETRO,NaN,NaN,NaN,NaN,NaN,NaN,35.061728,35.061728,34.671605,34.765432,500.0,67415.040000
5,1995-06-16,GRASIM,NaN,NaN,NaN,NaN,NaN,NaN,129.684348,129.684348,129.505061,129.505061,250.0,47011.311897
3,1995-06-16,BPCL,NaN,NaN,NaN,NaN,NaN,NaN,12.497917,12.497917,12.497917,12.497917,100.0,44992.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7593674,2025-06-13,NSLNISP,NaN,NaN,NaN,NaN,NaN,NaN,39.150000,39.420000,38.610000,39.050000,3150428.0,114440.158443
7592959,2025-06-13,GENUSPOWER,NaN,NaN,NaN,NaN,NaN,NaN,369.950000,382.800000,368.000000,370.500000,2990813.0,112605.359198
7594499,2025-06-13,WESTLIFE,-0.016727,0.278033,-0.025715,4.9878,-37.9719,35.366801,695.000000,718.950000,691.450000,716.050000,115027.0,111658.090948
7592639,2025-06-13,CCL,0.202749,NaN,0.490427,41.6672,38.9591,-0.342259,800.000000,841.050000,799.950000,833.700000,201301.0,111330.230424


In [52]:
merged_df = merged_df.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume', 'Mcap'])
merged_df

,Date,Symbol,Operating_profit_Growth,Advertising_Growth,Cash_from_Operation_Growth,Net_Sales_Growth,Adj_PAT_Growth,FCF_Growth
11,1995-06-16,RELIANCE,NaN,NaN,NaN,NaN,NaN,NaN
13,1995-06-16,TATASTEEL,NaN,NaN,NaN,NaN,NaN,NaN
8,1995-06-16,HINDPETRO,NaN,NaN,NaN,NaN,NaN,NaN
5,1995-06-16,GRASIM,NaN,NaN,NaN,NaN,NaN,NaN
3,1995-06-16,BPCL,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
7593674,2025-06-13,NSLNISP,NaN,NaN,NaN,NaN,NaN,NaN
7592959,2025-06-13,GENUSPOWER,NaN,NaN,NaN,NaN,NaN,NaN
7594499,2025-06-13,WESTLIFE,-0.016727,0.278033,-0.025715,4.9878,-37.9719,35.366801
7592639,2025-06-13,CCL,0.202749,NaN,0.490427,41.6672,38.9591,-0.342259


In [53]:
# Assuming your DataFrame is called `df`
# First, identify the columns to rank (excluding 'Date' and 'Symbol')
rank_cols = [col for col in merged_df.columns if col not in ['Date', 'Symbol']]

# Apply cross-sectional percentile rank for each date
merged_df[rank_cols] = merged_df.groupby('Date')[rank_cols].transform(lambda x: x.rank(pct=True))

merged_df

,Date,Symbol,Operating_profit_Growth,Advertising_Growth,Cash_from_Operation_Growth,Net_Sales_Growth,Adj_PAT_Growth,FCF_Growth
11,1995-06-16,RELIANCE,NaN,NaN,NaN,NaN,NaN,NaN
13,1995-06-16,TATASTEEL,NaN,NaN,NaN,NaN,NaN,NaN
8,1995-06-16,HINDPETRO,NaN,NaN,NaN,NaN,NaN,NaN
5,1995-06-16,GRASIM,NaN,NaN,NaN,NaN,NaN,NaN
3,1995-06-16,BPCL,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
7593674,2025-06-13,NSLNISP,NaN,NaN,NaN,NaN,NaN,NaN
7592959,2025-06-13,GENUSPOWER,NaN,NaN,NaN,NaN,NaN,NaN
7594499,2025-06-13,WESTLIFE,0.167476,0.609827,0.376499,0.300716,0.062972,0.985612
7592639,2025-06-13,CCL,0.485437,NaN,0.705036,0.906921,0.682620,0.328537


In [66]:
merged_df.to_csv('New Growth.csv', index=False)